In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import layers
import foolbox as fb

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = (x_train.reshape(60000, 784).astype('float32') / 255)
x = tf.convert_to_tensor(tf.expand_dims(x_train[422].reshape(28,28,1), axis=0), tf.float32)*1
y = tf.convert_to_tensor([y_train[422]])*1


class CustomLayer(layers.Layer):

    def __init__(self, units=32, activation='relu', input_shape=(784)):
        super(CustomLayer, self).__init__()
        self.units = units
        self.activation = activation
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True,
                                 name='unpruned-weights')
        self.mask = self.add_weight(shape=(self.w.shape),
                                    initializer='ones',
                                    trainable=False,
                                   name='pruning-masks')
        self.pruned_w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='ones',
                                 trainable=False,
                                       name='pruned-weights')

   
    def call(self, inputs):
        self.pruned_w = tf.multiply(self.w, self.mask)
        x = tf.matmul(inputs, self.pruned_w)
        
        if self.activation == 'relu':
            return tf.keras.activations.relu(x)
        if self.activation == 'softmax':
            return tf.keras.activations.softmax(x)
        if self.activation == None:
            return x
            
        raise ValueError('Activation function not implemented')


class LeNet300_100(tf.keras.Model):
    def __init__(self):
        super(LeNet300_100, self).__init__()
        self.dense1 = CustomLayer(300, input_shape=(None, 784))
        self.dense2 = CustomLayer(100, input_shape=(None, 300))
        self.dense3 = CustomLayer(10, activation=None, input_shape=(None, 100))
        
    def call(self, inputs):
        x = tf.keras.layers.Flatten()(inputs)
        x = self.dense1(x)
        
        x = self.dense2(x)
        x = self.dense3(x)
        self.pre_softmax = x
        return tf.keras.activations.softmax(x)

def initialize_model():
    model = LeNet300_100()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
                  metrics=['accuracy'],
                  experimental_run_tf_function=False
                 )
    return model

def train_model(model):
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    model.fit(x=x_train,
              y=y_train,
              batch_size=64,
              epochs=500,
              callbacks=[callback],
              validation_data=(x_test, y_test),
             )
    
model = initialize_model()
train_model(model)


#model = tf.keras.models.load_model('../saved-models/attack-test-model')
fmodel = fb.models.TensorFlowModel(model, bounds=(0,1))

attack = fb.attacks.L0BrendelBethgeAttack()

adversarials = attack(
    fmodel,
    x,
    y,
    epsilons=None
)

/Users/florianmerkle/dev/foolbox/foolbox/attacks/brendel_bethge.py:781: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec=[])
/Users/florianmerkle/dev/foolbox/foolbox/attacks/brendel_bethge.py:1364: NumbaDeprecationWarning: The 'numba.jitclass' decorator has moved to 'numba.experimental.jitclass' to better reflect the experimental nature of the functionality. Please update your imports to accommodate this change and see http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#change-of-jitclass-location for the time frame.
  @jitclass(spec=spec)
/Users/florianmerkle/dev/foolbox/foolbox/attacks/brendel_bethge.py:1560: NumbaDeprecationWarning: The 'numba.jitclass' decorator 

Train on 60000 samples, validate on 10000 samples
Epoch 1/500
60000/60000 [==============================] - 4s 64us/sample - loss: 1.5751 - accuracy: 0.8991 - val_loss: 1.5193 - val_accuracy: 0.9419
Epoch 2/500
60000/60000 [==============================] - 4s 62us/sample - loss: 1.5114 - accuracy: 0.9527 - val_loss: 1.5001 - val_accuracy: 0.9611
Epoch 3/500
60000/60000 [==============================] - 4s 63us/sample - loss: 1.4981 - accuracy: 0.9648 - val_loss: 1.4979 - val_accuracy: 0.9632
Epoch 4/500
60000/60000 [==============================] - 4s 63us/sample - loss: 1.4913 - accuracy: 0.9712 - val_loss: 1.4899 - val_accuracy: 0.9712
Epoch 5/500
60000/60000 [==============================] - 4s 68us/sample - loss: 1.4861 - accuracy: 0.9761 - val_loss: 1.4885 - val_accuracy: 0.9727
Epoch 6/500
60000/60000 [==============================] - 4s 66us/sample - loss: 1.4826 - accuracy: 0.9798 - val_loss: 1.4872 - val_accuracy: 0.9740
Epoch 7/500
60000/60000 [=========================

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Failed in nopython mode pipeline (step: nopython frontend)
Failed in nopython mode pipeline (step: nopython frontend)
Failed in nopython mode pipeline (step: nopython frontend)
Failed in nopython mode pipeline (step: nopython frontend)
Type of variable 'a_j.8' cannot be determined, operation: phi(incoming_values=[Var(a_j.9, brendel_bethge.py:1127), Undefined], incoming_blocks=[1518, 230]), location: /Users/florianmerkle/dev/foolbox/foolbox/attacks/brendel_bethge.py (1127)

File "../../foolbox/foolbox/attacks/brendel_bethge.py", line 1127:
    def _line_search_wolfe(  # noqa: C901
        <source elided>
                        (k == 0)
                        or (a_j is None)
                        ^

[1] During: resolving callee type: BoundFunction((<class 'numba.core.types.misc.ClassInstanceType'>, '_line_search_wolfe') for instance.jitclass.BFGSB#63c1f5710<>)
[2] During: typing of call at /Users/florianmerkle/d